* extract audio features out of audio files
* This notebook is inspired by the stage0_provided_information/gems-emotion-tags-main notebooks

# 0. pre stuff

In [16]:
# ! pip install essentia==2.1-beta6-dev
# ! pip install opensmile==2.4.1
# Installation:
# ! git clone https://github.com/openXBOW/openXBOW

In [43]:
# 1.
import glob

import essentia.standard as es

import shutil
import pandas as pd
import os

import json
import numpy as np


# 2.
import librosa as lr

import scipy.io.wavfile
from scipy import signal
import numpy as np

import opensmile 

# 3.
import re # to find digit pattern in column names
import openXBOW # compress time seq data 
from collections import defaultdict

from itertools import chain # to flatten a nested list

import BOW as bow

In [2]:
import sys
sys.path.insert(0, '../../stage-1/overall_used_tools')
import requirements_check as rc
import essentia, scipy, collections
rc.check(sys, [glob,essentia,shutil,pd,os,json,np,lr,scipy,np,opensmile,re,collections])

For 
glob 
python version needs to fit 
-> original version 3.7.3 
-> your version 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


Your version identical with original version 
-> essentia 
-> 2.1-beta6-dev


Possibly different versions: 
-> shutil 
-> original version 1.0.0 
-> your version  is not clearly visible. Go to your used python folder path .../python3.7/site-packages to investigate the version of your package. Also possible: have a look at the shell commands "pip show module_name" and "apt show module_name"


Your version identical with original version 
-> pandas 
-> 1.3.5


For 
os 
python version needs to fit 
-> original version 3.7.3 
-> your version 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


Your version identical with original version 
-> json 
-> 2.0.9


Your version identical with original version 
-> numpy 
-> 1.21.6


Your version identical with original version 
-> librosa 
-> 0.8.1


Your version identical with original version 
-> scipy 
-> 1.7

# 1. essentia --> extract from files
Works with .wav<br>
Getting output matrix for low level extractor features with shape (time, feature) --> When not using CNN/LSTM use its statistics and/or openXBow which creates histogram. <br>
Info:

In [3]:
help(es.MusicExtractor)

Help on class Algo in module essentia.standard:

class Algo(Algorithm)
 |  Algo(**kwargs)
 |  
 |  MusicExtractor
 |  
 |  
 |  Inputs:
 |  
 |    [string] filename - the input audiofile
 |  
 |  
 |  Outputs:
 |  
 |    [pool] results - Analysis results pool with across-frames statistics
 |    [pool] resultsFrames - Analysis results pool with computed frame values
 |  
 |  
 |  Parameters:
 |  
 |    analysisSampleRate:
 |      real ∈ (0,inf) (default = 44100)
 |      the analysis sampling rate of the audio signal [Hz]
 |  
 |    chromaprintCompute:
 |      bool ∈ {true,false} (default = false)
 |      compute the Chromaprint
 |  
 |    chromaprintDuration:
 |      real ∈ [0,inf) (default = 0)
 |      the amount of time from the beginning used to compute the Chromaprint. 0 to
 |      use the full audio length [s]
 |  
 |    endTime:
 |      real ∈ [0,inf) (default = 1000000)
 |      the end time of the slice you want to extract [s]
 |  
 |    gfccStats:
 |      vector_string (default 

### 1.1 Compute mean and stdev for all features


In [3]:
# extracting all features
my_dir = '../../stage1_data_collecting_phase/audio2midi_converter/track_preparation/accompaniment/'# os.getcwd()
data  = my_dir #+ '/.wav' # ! path to wav files
sel_features = {'Song': []}  # Dic to collect float-like and 1D-array (except rhythm.beats_position) features and put them in a csv
all_features = {}  # Dic to collect all the features disregarding their object type

df_music_ex = pd.DataFrame()

for file in glob.glob(os.path.join(data, '*.wav')):
        
    file_name  = os.path.basename(file[0:-4])    
    
    sel_features['Song'].append(file_name) # ! select only features with single float values
    all_features[file_name] = {}
    # Compute all features, aggregate only 'mean' and 'stdev' statistics for all low-level, rhythm and tonal frame features
    features, features_frames = es.MusicExtractor(lowlevelStats=["mean", "var", "stdev", "median", "min", "max"],
                                                  rhythmStats=["mean", "var", "stdev", "median", "min", "max"],
                                                  tonalStats=["mean", "var", "stdev", "median", "min", "max"])(file)
   
    features_names = sorted(features.descriptorNames())
    for f in features_names:
        all_features[file_name][f] = features[f]
        if type(features[f]) is float:
            if f not in sel_features:
                sel_features[f] = [features[f]]
            else:
                sel_features[f].append(features[f])
        
        elif type(features[f]) is not str:
            if len(features[f].shape) == 1 and f != 'rhythm.beats_position':
                for index, elem in enumerate(features[f]):
                    new_name = f + '_' + str(index)
                    if new_name not in sel_features:
                        sel_features[new_name] = [elem]
                    else:
                        sel_features[new_name].append(elem)

df_music_ex = pd.DataFrame(sel_features)
# fit the index format to the ones of functionals which will follow:
df_music_ex.reset_index(drop=True, inplace=True)
idx = pd.Index(df_music_ex['Song']+'.wav', name='file')
del df_music_ex['Song']
df_music_ex = df_music_ex.set_index([idx])


[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, a

[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, a

[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, a

[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, a

[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, a

...

[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, a

[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, a

[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, a

[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, a

[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, a

### 1.2 Save selected features as csv file -> single float values

In [21]:
''''
os.makedirs('RESULTS', exist_ok=True)
df_music_ex.to_csv('RESULTS/audio_scalarFeat_essentia.csv', index=True)
'''

**NOTE:** I have ignored rhythm.beats_position in the csv because the length of arrays varies across songs and makes problems when fitting them in a table. Categorical variables are also ignored in the csv.

# 2. Open Smile --> extract from files features
Works with .wav. 

## 2.1 emobase
Drop feature Probability of Voice.

In [ ]:
# !pip install opensmile==2.4.1

# When problems in installing opensmile because of existing packages which are not deletable, use:
# !pip install opensmile --ignore-installed
# Then update opensmile:
# !pip install opensmile -U

### 2.1.1 Extracting emobase LLDs (low-level descriptors) from several audio files
2d array for each file/sample

In [ ]:
my_dir = '../../stage1_data_collecting_phase/audio2midi_converter/track_preparation/accompaniment/'
data  = my_dir

# Define feature extractor to get LLDs from the eGeMAPS (v02) feature set
smile_LLDs = opensmile.Smile(
    feature_set=opensmile.FeatureSet.emobase,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

# Create a folder to save the results
os.makedirs('RESULTS/LLD_emobase', exist_ok=True)

# Extract the features
lld_df = pd.DataFrame()
for file in glob.glob(os.path.join(data, '*.wav')):
    file_name  = os.path.basename(file[0:-4])
    LLDs = smile_LLDs.process_file(file) # LLDs as pd of shape (time frames of single file,features)

    # remove indices and replace them with same ones as for bow_df, e.g. 'b.wav'
    LLDs.reset_index(drop=True, inplace=True)
    idx = pd.Index([f"'{file_name}.wav'"]*LLDs.shape[0],name='name')
    LLDs = LLDs.set_index([ idx])

    
    
    lld_df = lld_df.append(LLDs)
    

# delete unwanted feature:
del lld_df['voiceProb_sma']

        
# Create csv file for bow dataframe:
csv_name_llds = 'RESULTS/LLD_emobase/' + 'full_bow_lld' + '.csv'
lld_df.to_csv(csv_name_llds, index=True)  # remember to drop the index   # for each file a .csv generated
lld_df.head()   

In [ ]:
# Store file:
'''
csv_name_llds = 'RESULTS/LLD_emobase/' + 'full_bow_lld' + '.csv'
lld_df.to_csv(csv_name_llds, index=True)  # remember to drop the index   # for each file a .csv generated
lld_df.head()   
'''

**LLDs gives us values of very tiny time slots. Several time slots together are around 45 seconds and lead to the summarizing feature description feature[time_zone]. Therefore, later we want to keep several time zone and calculate for each the bow. Because of the time split precision it does not fit to the functionals. Make an extra bow. Also scalar features are splitted in time slots and therefore usable for BoW, too.**

### 2.1.2 Extracting emobase functionals from several audio files
simple features

In [27]:
# Define feature extractor to get functionals from the eGeMAPS (v02) feature set
smile_func = opensmile.Smile(
    feature_set=opensmile.FeatureSet.emobase,
    feature_level=opensmile.FeatureLevel.Functionals,
)

# get list of paths for all the files
all_files = glob.glob(os.path.join(data, '*.wav'))

# extract functionals for all the files in a dataframe
functionals1 = smile_func.process_files(all_files)

# clean up the index
functionals1.reset_index(inplace=True)
functionals1['file'] = functionals1['file'].apply(os.path.basename)
try:
    del functionals1['start']
    del functionals1['end']
except:
    pass
functionals1.set_index(['file'], inplace=True)
c1 = functionals1.columns
# remove unneeded features with 'voiceProb':
func1_list = functionals1.columns.tolist()
for ele in func1_list:
    if 'voiceProb' in ele:
        del functionals1[ele]

# save data frame to csv:
functionals1.to_csv('RESULTS/functionals_emobase.csv', index=True)  # now we keep the index since we will need the file name as we have more than one 
# csv contains all files
functionals1.head()

,pcm_intensity_sma_max,pcm_intensity_sma_min,pcm_intensity_sma_range,pcm_intensity_sma_maxPos,pcm_intensity_sma_minPos,pcm_intensity_sma_amean,pcm_intensity_sma_linregc1,pcm_intensity_sma_linregc2,pcm_intensity_sma_linregerrA,pcm_intensity_sma_linregerrQ,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
file,,,,,,,,,,,,,,,,,,,,,
P_EllieGoulding_Burn_accompaniment.wav,0.000007,0.0,0.000007,2709.0,0.0,9.942648e-07,1.156728e-10,8.179796e-07,7.149870e-07,8.684310e-13,...,6.536409,2.556911,10.470149,188.103104,-0.117249,0.0,0.000000,0.117249,0.000000,0.117249
H_KRS-One_SoundOf_accompaniment.wav,0.000013,0.0,0.000013,2441.0,0.0,8.221909e-07,-3.145658e-11,8.851198e-07,9.449687e-07,1.927819e-12,...,8.045993,2.836550,4.810075,84.599968,-0.020110,0.0,0.000000,0.020110,0.000000,0.020110
P_TameImpala_TheLess_accompaniment.wav,0.000009,0.0,0.000009,3399.0,0.0,1.165094e-06,-3.573224e-11,1.259016e-06,9.427505e-07,1.645060e-12,...,10.445717,3.232165,0.250710,20.534397,-0.337449,0.0,0.154327,0.337449,0.154327,0.491776
K_Beethoven_Mondsch_accompaniment.wav,0.000011,0.0,0.000011,652.0,0.0,7.904749e-07,-6.787946e-11,9.431018e-07,6.553323e-07,1.088981e-12,...,6.243113,2.499023,3.120666,39.210262,-0.319376,0.0,0.161715,0.319376,0.161715,0.481091
K_Wagner_Tristan_accompaniment.wav,0.000005,0.0,0.000005,2415.0,0.0,4.139789e-07,9.155792e-12,3.906408e-07,3.418292e-07,2.645131e-13,...,13.754009,3.708749,3.194038,44.686558,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


## 2.2 Extracting ComParE functionals from several audio files
simple features

In [28]:
# Define feature extractor to get functionals from the ComParE feature set
smile_func = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)

# get list of paths for all the files
all_files = glob.glob(os.path.join(data, '*.wav'))

# extract functionals for all the files in a dataframe
functionals2 = smile_func.process_files(all_files)

# clean up the index
functionals2.reset_index(inplace=True)
functionals2['file'] = functionals2['file'].apply(os.path.basename)
try:
    del functionals2['start']
    del functionals2['end']
except:
    pass
functionals2.set_index(['file'], inplace=True)
c2 = functionals2.columns
# remove unneeded features with 'voiceProb':
func2_list = functionals2.columns.tolist()
for ele in func2_list:
    if 'voicing' in ele:
        del functionals2[ele]

# save data frame to csv:
functionals2.to_csv('RESULTS/functionals_compare.csv', index=True)  # now we keep the index since we will need the file name as we have more than one 
functionals2.head()

,audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,audspec_lengthL1norm_sma_percentile1.0,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
file,,,,,,,,,,,,,,,,,,,,,
P_EllieGoulding_Burn_accompaniment.wav,3.044510,0.814900,0.0,0.923988,1.294802,1.643223,0.370813,0.348421,0.719235,0.019125,...,6.465429,0.556869,1.945888,1.945749,20.0,0.585743,94.102280,42.978065,93.732109,44.584675
H_KRS-One_SoundOf_accompaniment.wav,3.184608,0.648500,0.0,0.570475,0.795003,1.135774,0.224528,0.340770,0.565298,0.015332,...,8.033433,0.551020,2.236733,2.236733,20.0,0.543250,104.360443,54.235817,107.240715,57.155605
P_TameImpala_TheLess_accompaniment.wav,3.083809,0.402588,0.0,0.748631,1.140326,1.550072,0.391694,0.409746,0.801440,0.023902,...,8.871999,0.584771,2.241283,2.241283,-20.0,0.582534,106.933456,58.748146,103.683899,56.746696
K_Beethoven_Mondsch_accompaniment.wav,2.366065,0.727758,0.0,0.740924,0.900181,1.093534,0.159257,0.193354,0.352610,0.012854,...,5.349102,0.439836,1.396391,1.396958,-20.0,0.629886,57.708839,31.652971,57.702095,32.560139
K_Wagner_Tristan_accompaniment.wav,2.283237,0.512458,0.0,1.049790,1.363399,1.545837,0.313609,0.182438,0.496047,0.016960,...,7.983775,0.591497,2.275579,2.275579,-20.0,0.536667,105.867485,52.255013,104.461945,49.454681


In [ ]:
# ?opensmile.FeatureSet.ComParE_2016

# 3. MfCCs --> Modifying what we have
aggregate with BoAw (Bag of Audio words) of openXBOW. <br>
= Mel Frequency Cepstral Coefficients. Already included in essentia and opensmile's emobase and ComParE.

## 3.1 Create BoW

### 3.1.1 ... for functionals

In [31]:
# functionals (emobase and ComParE) have part-wise same column titles but different values 
# (which can be quite different). Therefore it could make sense to take both:

functionals2.columns += '_f2' # append column names to avoid duplicated names with functionals1
# when choosing only one then functionals2 becuase it has smaller values, not overwhelming big
# (with .drop as below)
both_functionals = pd.concat([functionals1, functionals2, df_music_ex],axis=1)

In [46]:
bow_class = bow.BOWConverter(None)
bow_functionals_df, time_interval_dict = bow_class.bow_executer(both_functionals)

CALL IT TO START WHOLE PROCEDURE, INPUT A ALREADY FITTING DATAFRAME WHICH HAS COLUMN TITLES WHERE A FEATURE IS TIME FRAMED BY: FEATURE[t]ONGINGDESCRIPTION OR FEATURE_t WHERE 't' DESCRIBES CURRENT TIME POINT 
We have extracted 26 BoAW-features from 2 files


Error: Unable to access jarfile ../../stage-1/overall_used_tools/openXBOW/openXBOW.jar
Error: Unable to access jarfile ../../stage-1/overall_used_tools/openXBOW/openXBOW.jar


### 3.1.2 ... for LLDs

In [47]:
bow_llds_df, _ = bow_class.bow_executer(lld_df, True)

CALL IT TO START WHOLE PROCEDURE, INPUT A ALREADY FITTING DATAFRAME WHICH HAS COLUMN TITLES WHERE A FEATURE IS TIME FRAMED BY: FEATURE[t]ONGINGDESCRIPTION OR FEATURE_t WHERE 't' DESCRIBES CURRENT TIME POINT 


KeyboardInterrupt: 

## 3.2 Collecting only scalar features for functionals

In [48]:
time_val_list = list(time_interval_dict.values())
flat_list = list(chain(*time_val_list)) # flatten list

NameError: name 'time_interval_dict' is not defined

In [ ]:
scalar_functional_df = both_functionals.drop(flat_list, axis=1) # drop all time-seq features
scalar_functional_df.index.names = ['name']
#scalar_functional_df.rename(index={scalar_functional_df.index: scalar_functional_df.index+'s'},inplace=True)
scalar_functional_df.index = "'" + scalar_functional_df.index + "'"  # we change indices value names 
# from, e.g. b.wav to 'b.wav'. So we can combine this dataframe with the bow_functionals_df from 3.1 in 3.3.
scalar_functional_df.head()

## 3.3 Bring BOW dataframes of functionals and LLDs (3.1) and scalar dataframe (3.2) together

In [50]:
bow_llds_df.columns += '_lld' # change column titles such that they don't get duplicated when fusion below

final_audio_feat_scalar_bow_df = pd.concat([bow_functionals_df, scalar_functional_df, bow_llds_df], axis=1)

In [51]:
# store the final scalar, bow feature dataframe in a file:
csv_name = 'RESULTS/audio_ScalarFeat_BOWFeat.csv'
final_audio_feat_scalar_bow_df.to_csv(csv_name, index=True) # index=False because numeric index gets already created


In [52]:
pd.read_csv(csv_name).head()

,name,W(0),W(1),W(2),W(3),W(4),W(5),W(6),W(7),W(8),...,W(16),W(17),W(18),W(19),W(20),W(21),W(22),W(23),W(24),W(25)
0,'0',1.079181,0.90309,1.041393,0.845098,1.113943,0.954242,0.845098,1.113943,1.041393,...,0.69897,0.778151,1.146128,1.041393,1.113943,1.079181,1.176091,1.113943,1.113943,1.113943
1,'1',1.079181,0.90309,1.041393,0.845098,1.113943,0.954242,0.845098,1.113943,1.041393,...,0.69897,0.778151,1.146128,1.041393,1.113943,1.079181,1.176091,1.113943,1.113943,1.113943
